In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from datetime import datetime
from tensorflow.keras.optimizers import Adam

In [2]:
# Define paths
data_train_path = 'Fabrics/train'
data_test_path = 'Fabrics/test'
data_val_path = 'Fabrics/validate'

# Image dimensions
img_width = 180
img_height = 180


In [3]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

# Load datasets with normalization
normalization_layer = layers.Rescaling(1./255)

data_train = tf.keras.utils.image_dataset_from_directory(
    data_train_path,
    shuffle=True,
    image_size=(img_width, img_height),
    batch_size=32
).map(lambda x, y: (normalization_layer(x), y))


data_val = tf.keras.utils.image_dataset_from_directory(
    data_val_path,
    shuffle=False,
    image_size=(img_width, img_height),
    batch_size=32
).map(lambda x, y: (normalization_layer(x), y))

data_test = tf.keras.utils.image_dataset_from_directory(
    data_test_path,
    shuffle=False,
    image_size=(img_width, img_height),
    batch_size=32
).map(lambda x, y: (normalization_layer(x), y))


Found 2292 files belonging to 22 classes.
Found 2284 files belonging to 22 classes.
Found 2288 files belonging to 22 classes.


In [4]:
# Define the model
base_model = ResNet50(input_shape=(img_width, img_height, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(22, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50 (Functional)                │ (None, 6, 6, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ ?                           │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=0.1, restore_best_weights=True)

history = model.fit(
    data_train,
    validation_data=data_val,
    epochs=5,
    callbacks=[early_stopping]
)

Epoch 1/5
72/72 ━━━━━━━━━━━━━━━━━━━━ 266s 4s/step - accuracy: 0.2080 - loss: 2.5627 - val_accuracy: 0.0079 - val_loss: 3.6987
Epoch 2/5
 2/72 ━━━━━━━━━━━━━━━━━━━━ 2:00 2s/step - accuracy: 0.1797 - loss: 2.3891    

In [ ]:
# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()


In [9]:
test_loss, test_acc = model.evaluate(data_test, verbose=2)
print(f'Test accuracy: {test_acc}')

72/72 - 121s - 2s/step - accuracy: 0.0210 - loss: 3.3975
Test accuracy: 0.02097902074456215


In [10]:
model_save_path = 'garmet_model1.keras'
model.save(model_save_path)
print(f'Model saved to {model_save_path}')

Model saved to garmet_model1.keras
